In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

C:\Users\LASYA\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
data = pd.read_csv("data.csv")
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:, 0:1024]
Y = Y[:, 1024]
X_train = X[0:70000, :]
X_train = X_train / 255.
X_test = X[70000:72001, :]
X_test = X_test / 255.
# Reshape
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:70000, :]
Y_train = Y_train.T
Y_test = Y[70000:72001, :]
Y_test = Y_test.T

In [3]:
print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

number of training examples = 70000
number of test examples = 2000
X_train shape: (70000, 1024)
Y_train shape: (1, 70000)
X_test shape: (2000, 1024)
Y_test shape: (1, 2000)


In [4]:
image_x = 32
image_y = 32

train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test)
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0], 32, 32, 1)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 1)

In [5]:
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(train_y.shape))

X_train shape: (70000, 32, 32, 1)
Y_train shape: (70000, 37)


In [6]:
#Building model
def keras_model(image_x,image_y):
    num_of_classes = 37
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "devanagari.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]

    return model, callbacks_list


In [7]:
model, callbacks_list = keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=8, batch_size=64,callbacks=callbacks_list)
scores = model.evaluate(X_test, test_y, verbose=0)
print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
print_summary(model)
model.save('devanagari.h5')

W0815 13:43:10.554732 10700 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 13:43:14.619377 10700 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 13:43:15.528964 10700 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 13:43:16.692497 10700 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0815 13:43:17.080791 10700 deprecation_wrapper.py:119] From C:\Users\LASYA\Ana

Train on 70000 samples, validate on 2000 samples
Epoch 1/8
70000/70000 [==============================] - 107s 2ms/step - loss: 0.7642 - acc: 0.7845 - val_loss: 0.2939 - val_acc: 0.9040

Epoch 00001: val_acc improved from -inf to 0.90400, saving model to devanagari.h5
Epoch 2/8
70000/70000 [==============================] - 96s 1ms/step - loss: 0.2268 - acc: 0.9315 - val_loss: 0.2175 - val_acc: 0.9255

Epoch 00002: val_acc improved from 0.90400 to 0.92550, saving model to devanagari.h5
Epoch 3/8
70000/70000 [==============================] - 92s 1ms/step - loss: 0.1485 - acc: 0.9551 - val_loss: 0.1373 - val_acc: 0.9535

Epoch 00003: val_acc improved from 0.92550 to 0.95350, saving model to devanagari.h5
Epoch 4/8
70000/70000 [==============================] - 93s 1ms/step - loss: 0.1099 - acc: 0.9661 - val_loss: 0.1270 - val_acc: 0.9650

Epoch 00004: val_acc improved from 0.95350 to 0.96500, saving model to devanagari.h5
Epoch 5/8
70000/70000 [==============================] - 92s 1ms/